# Applied Data Science Capstone final project.


This notebook is my final project for the Applied Data Science Capstone course from Coursera. In this notebook, a location problem will be studied.

### Problem description.

One of the most beautiful parks in Chicago,IL, is the Millennium Park and he is also one of the most positively reviewed. The park is one of the most famous parks in IL, attracting millions of visitors a year.

Due the large number of visitors the park receives every day, an investor would like to know if the surrounding of the park would be a good place to build his restaurant. To answer this question, the investor will open his restaurant if there is a region, next to the park, with not too many competitors. If such area is available, he also wants to know if it is better to open an Italian, a vegetarian or a Japanese restaurant, the types of food he already invests in.

### research methodology.

To answer the question proposed in the business problem, some location information will be required. First, it is necessary the latitude and longitude of the park. Due to its size, the center of the park will be used as reference for its location. The latitude and longitude were already identified using Google maps and is 41.882222, -87.621370. With the latitude and longitude of the park, the closest food venues will be identified through Foursquare API, with a radius of 2500 meters. This large radius was chosen due the size of the park. With a 2500 radius, we will assure that at least a five-block radius surrounding the park will be considered when looking for food venues.

After collecting the data, the data will be reviewed and cleaned. A cluster analysis will be performed using the latitude and longitude of the venues to identify regions with lower density of food venues. The clusters will be plotted through using folium to make the visualization easier. If there is a low density region, the categories of the venues will be reviewed to see what are the most popular food types and a recommendation of a type of restaurant will be made based on the other types of restaurants available in the region selected.


### code with explanation:

1- Importing all the libraries needed for out project.

In [46]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed
Libraries imported.


2-Connect to Foursquare API.

In [47]:
CLIENT_ID = 'W4TBYFIZOQYOS0XBZOPJUNPDNFYCF42OZDP1QPIEEJZG52I4' # your Foursquare ID
CLIENT_SECRET = 'QFLLGATP1YPVVHGI2EZTVVV3RP54CZFGKOZR343JJKJRRKD2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W4TBYFIZOQYOS0XBZOPJUNPDNFYCF42OZDP1QPIEEJZG52I4
CLIENT_SECRET:QFLLGATP1YPVVHGI2EZTVVV3RP54CZFGKOZR343JJKJRRKD2


3-Defining our search query for the Foursquare API while considering these:
A)the search query will be food since we are interested in opening a restaurant.

B) The minimum radius will be 2500 meter due the size of the park.

C) The latitude and longitude of the center of the park were identified through Google Maps.

In [48]:
#First we define what we are looking for
search_query = 'Food'

#Then we define the radious
radius = 2500

#Get the latitude and longitude Ibirapuera park 
lat = 41.882222
long = -87.621370

#Finaly we set the url for our search
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)

4-Requesting the results from the search query

In [49]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dbf19edd03360002cb93af7'},
 'response': {'venues': [{'id': '4c22665899282d7f411067b0',
    'name': 'Atrium Food Court',
    'location': {'address': '100 W Randolph, Concourse',
     'crossStreet': 'btw Clark & Lake',
     'lat': 41.88516275046533,
     'lng': -87.63163902479333,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.88516275046533,
       'lng': -87.63163902479333}],
     'distance': 911,
     'postalCode': '60601',
     'cc': 'US',
     'city': 'Chicago',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['100 W Randolph, Concourse (btw Clark & Lake)',
      'Chicago, IL 60601',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d120951735',
      'name': 'Food Court',
      'pluralName': 'Food Courts',
      'shortName': 'Food Court',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_foodcourt_',
       'suffix': '.png'},
      'primary': True}],
    'ven

5-Assigning the .json file to a dataframe.

In [50]:
venues = results['response']['venues']

# tranform venues into a dataframe
food_venues = json_normalize(venues)
food_venues.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood
0,4c22665899282d7f411067b0,Atrium Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1572805101,False,"100 W Randolph, Concourse",btw Clark & Lake,41.885163,-87.631639,"[{'label': 'display', 'lat': 41.88516275046533...",...,United States,"[100 W Randolph, Concourse (btw Clark & Lake),...",57973203,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b1695b8f964a52094ba23e3,Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1572805101,False,520 N Michigan Ave,at The Shops at North Bridge,41.892102,-87.626163,"[{'label': 'display', 'lat': 41.8921018, 'lng'...",...,United States,[520 N Michigan Ave (at The Shops at North Bri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5751af74cd102f915c2cb3de,Revival Food Hall,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1572805101,False,125 S Clark St,W Adams St,41.879560,-87.630550,"[{'label': 'display', 'lat': 41.87956, 'lng': ...",...,United States,"[125 S Clark St (W Adams St), Chicago, IL 6060...",NaN,384562,https://www.grubhub.com/restaurant/farmers-fri...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
3,500f3f21e4b09c8ae488d96e,Navy Pier Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1572805101,False,800 E Grand Ave,Navy Pier,41.891489,-87.608547,"[{'label': 'display', 'lat': 41.89148923128558...",...,United States,"[800 E Grand Ave (Navy Pier), Chicago, IL 6061...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4b8aac5bf964a520f97832e3,Bubblenet Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1572805101,False,1200 S Lake Shore Dr,@ Shedd Aquarium,41.867925,-87.613260,"[{'label': 'display', 'lat': 41.86792457273813...",...,United States,"[1200 S Lake Shore Dr (@ Shedd Aquarium), Chic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


6-Filtering our dataframe to keep only the relevant parts.

In [51]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in food_venues.columns if col.startswith('location.')] + ['id']
food_filtered = food_venues.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
food_filtered['categories'] = food_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
food_filtered.columns = [column.split('.')[-1] for column in food_filtered.columns]

food_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Atrium Food Court,Food Court,"100 W Randolph, Concourse",btw Clark & Lake,41.885163,-87.631639,"[{'label': 'display', 'lat': 41.88516275046533...",911,60601,US,Chicago,IL,United States,"[100 W Randolph, Concourse (btw Clark & Lake),...",NaN,4c22665899282d7f411067b0
1,Food Court,Food Court,520 N Michigan Ave,at The Shops at North Bridge,41.892102,-87.626163,"[{'label': 'display', 'lat': 41.8921018, 'lng'...",1169,60611,US,Chicago,IL,United States,[520 N Michigan Ave (at The Shops at North Bri...,NaN,4b1695b8f964a52094ba23e3
2,Revival Food Hall,Food Court,125 S Clark St,W Adams St,41.879560,-87.630550,"[{'label': 'display', 'lat': 41.87956, 'lng': ...",816,60603,US,Chicago,IL,United States,"[125 S Clark St (W Adams St), Chicago, IL 6060...",NaN,5751af74cd102f915c2cb3de
3,Navy Pier Food Court,Food Court,800 E Grand Ave,Navy Pier,41.891489,-87.608547,"[{'label': 'display', 'lat': 41.89148923128558...",1481,60611,US,Chicago,IL,United States,"[800 E Grand Ave (Navy Pier), Chicago, IL 6061...",NaN,500f3f21e4b09c8ae488d96e
4,Bubblenet Food Court,Food Court,1200 S Lake Shore Dr,@ Shedd Aquarium,41.867925,-87.613260,"[{'label': 'display', 'lat': 41.86792457273813...",1727,60605,US,Chicago,IL,United States,"[1200 S Lake Shore Dr (@ Shedd Aquarium), Chic...",NaN,4b8aac5bf964a520f97832e3
5,Aztec Daves Food Truck,Food Truck,150 s Wacker dr,Adams,41.883796,-87.657836,"[{'label': 'display', 'lat': 41.88379593996464...",3027,60606,US,Chicago,IL,United States,"[150 s Wacker dr (Adams), Chicago, IL 60606, U...",NaN,555f60e6498ed73c7460f908
6,Food For Thought - Tribune Tower,American Restaurant,435 N Michigan Ave,NaN,41.890609,-87.623482,"[{'label': 'display', 'lat': 41.89060943463509...",949,60611,US,Chicago,IL,United States,"[435 N Michigan Ave, Chicago, IL 60611, United...",NaN,4b8292caf964a520f0d830e3
7,Marina Food & Liquor,Grocery Store,300 N State St,NaN,41.887243,-87.629308,"[{'label': 'display', 'lat': 41.88724300512964...",863,60654,US,Chicago,IL,United States,"[300 N State St, Chicago, IL 60654, United Sta...",NaN,4c41dc163735be9aee0619a4
8,Great Food Tours,Moving Target,NaN,NaN,41.890104,-87.628180,"[{'label': 'display', 'lat': 41.8901037281748,...",1043,60611,US,Chicago,IL,United States,"[Chicago, IL 60611, United States]",NaN,4fc26d68e4b09af80b7cd3ff
9,Whole Foods Market,Grocery Store,1101 S. Canal St,btwn Roosevelt & Taylor St,41.868010,-87.638427,"[{'label': 'display', 'lat': 41.86801034360315...",2121,60607,US,Chicago,IL,United States,[1101 S. Canal St (btwn Roosevelt & Taylor St)...,NaN,49da4033f964a520625e1fe3


7- Now the ID's and names are easier to interpret. When we look to the categories of the venues, some of them are related to food, but are not restaurants. The venue 14, for example, is a office.and Therefore, it is necessary to clean the dataframe.

Data cleaning.
To clean the framework, the categories column will be used. If the words "restaurant", "cafe" or "Food" are not presented in the categories, it is likely that the venue will not be a competitor to our investor, so this entries will be droped.

In [53]:
#Creating an empty list that will keep the venues to be droped
not_restaurant = []

#Checking if the words 'restaurant', 'food' or 'cafe' are not in the column categories
for i in range(len(food_filtered['categories'])):
    if 'Restaurant' not in food_filtered['categories'].iloc[i]:
        if 'Food' not in food_filtered['categories'].iloc[i]:
            if 'Cafe' not in food_filtered['categories'].iloc[i]:
                not_restaurant.append(i)

#Dropping the row identified in the for loop
food_filtered.drop(food_filtered.index[[not_restaurant]], axis=0, inplace=True)
food_filtered.head(15)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Atrium Food Court,Food Court,"100 W Randolph, Concourse",btw Clark & Lake,41.885163,-87.631639,"[{'label': 'display', 'lat': 41.88516275046533...",911,60601,US,Chicago,IL,United States,"[100 W Randolph, Concourse (btw Clark & Lake),...",NaN,4c22665899282d7f411067b0
1,Food Court,Food Court,520 N Michigan Ave,at The Shops at North Bridge,41.892102,-87.626163,"[{'label': 'display', 'lat': 41.8921018, 'lng'...",1169,60611,US,Chicago,IL,United States,[520 N Michigan Ave (at The Shops at North Bri...,NaN,4b1695b8f964a52094ba23e3
2,Revival Food Hall,Food Court,125 S Clark St,W Adams St,41.879560,-87.630550,"[{'label': 'display', 'lat': 41.87956, 'lng': ...",816,60603,US,Chicago,IL,United States,"[125 S Clark St (W Adams St), Chicago, IL 6060...",NaN,5751af74cd102f915c2cb3de
3,Navy Pier Food Court,Food Court,800 E Grand Ave,Navy Pier,41.891489,-87.608547,"[{'label': 'display', 'lat': 41.89148923128558...",1481,60611,US,Chicago,IL,United States,"[800 E Grand Ave (Navy Pier), Chicago, IL 6061...",NaN,500f3f21e4b09c8ae488d96e
4,Bubblenet Food Court,Food Court,1200 S Lake Shore Dr,@ Shedd Aquarium,41.867925,-87.613260,"[{'label': 'display', 'lat': 41.86792457273813...",1727,60605,US,Chicago,IL,United States,"[1200 S Lake Shore Dr (@ Shedd Aquarium), Chic...",NaN,4b8aac5bf964a520f97832e3
5,Aztec Daves Food Truck,Food Truck,150 s Wacker dr,Adams,41.883796,-87.657836,"[{'label': 'display', 'lat': 41.88379593996464...",3027,60606,US,Chicago,IL,United States,"[150 s Wacker dr (Adams), Chicago, IL 60606, U...",NaN,555f60e6498ed73c7460f908
6,Food For Thought - Tribune Tower,American Restaurant,435 N Michigan Ave,NaN,41.890609,-87.623482,"[{'label': 'display', 'lat': 41.89060943463509...",949,60611,US,Chicago,IL,United States,"[435 N Michigan Ave, Chicago, IL 60611, United...",NaN,4b8292caf964a520f0d830e3
10,Chicago Union Station Food Court,Food Court,225 S Canal St,at Adams St.,41.879339,-87.638738,"[{'label': 'display', 'lat': 41.879339217638, ...",1474,60606,US,Chicago,IL,United States,"[225 S Canal St (at Adams St.), Chicago, IL 60...",NaN,4c489f3a19fde21e8b460c76
11,Bridgeport Pasty - Food Truck,Food Truck,NaN,NaN,41.879691,-87.636715,"[{'label': 'display', 'lat': 41.87969136866029...",1302,60608,US,Chicago,IL,United States,"[Chicago, IL 60608, United States]",NaN,4e8c8fc793adecfdb971a040
12,"Hello, Food - Groupon",Cafeteria,600 W Chicago Ave,at N Larrabee St,41.897139,-87.644433,"[{'label': 'display', 'lat': 41.89713919135436...",2531,60654,US,Chicago,IL,United States,"[600 W Chicago Ave (at N Larrabee St), Chicago...",NaN,50097e09e4b0b96fdb0d3015


As we can see, the row 14 and all other rows that didn't contained the words 'Restaurant', 'Cafe' or 'Food' we droped, therefore, the dataframe is clean.
8- Explore the location of the venues using their latitude and longitude.
To explore such locations, let's use the folium module to plot them.

In [54]:
venues_map = folium.Map(location=[lat, long], zoom_start=15) # generate map centred around Millennium Park

# add a orange circle marker to represent Millennium Park
folium.features.CircleMarker(
    location = [lat, long],
    radius=10,
    color='red',
    popup='Millennium Park',
    fill = True,
    fill_color = 'Yellow',
    fill_opacity = 0.6
).add_to(venues_map)

# add the other restaurants and food places as blue circle markers
for lat, lng, label in zip(food_filtered.lat, food_filtered.lng, food_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Research method

To further analyze the food venues in the region, a cluster analysis will be used to group the venues based on their latitude and longitude. To perform the clustering, Kmeans will be used. Kmeans was chosen because it is a simple algorithm to be implemented and will have a good performance considering the number of data points.

After forming the clusters, the region that doesn’t present a cluster, or presents the cluster with the lowest density, will be chosen to be the potential location. With the potential location in mind, the surrounding venues of the location will be evaluated. If there is a region with no venues, the closest cluster will be used to choose the appropriate restaurant. Since our investor wants to open a restaurant with fewer competitors, the categories of the venue will be reviewed to see how many Japanese, Italian and vegetarian restaurants are in the region, the type of restaurant that has fewer direct competitors will be chosen.

1- load an extra module.

In [55]:
from sklearn.cluster import KMeans

2-let's initialize a k_means algorithm,Since we have a large radius, let's use six clusters for the analysis.
and we will use the latitude and longitude of the venues to cluster them.

In [56]:
#Initializing k-means with seven clusters
k_means = KMeans(init="k-means++", n_clusters=6, n_init=12)

#Grouping the location based on latitude and longitude
k_means.fit(food_filtered[['lat','lng']])

#Get the label of the cluster
k_means_labels = k_means.labels_
k_means_labels

array([1, 2, 1, 0, 5, 4, 0, 1, 1, 3, 2, 2, 1, 2, 1, 2, 1, 1, 3, 0, 0, 0],
      dtype=int32)

3-add a column in the dataframe to store the cluster numbers.

In [57]:
food_filtered['cluster'] = k_means_labels
food_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,cluster
0,Atrium Food Court,Food Court,"100 W Randolph, Concourse",btw Clark & Lake,41.885163,-87.631639,"[{'label': 'display', 'lat': 41.88516275046533...",911,60601,US,Chicago,IL,United States,"[100 W Randolph, Concourse (btw Clark & Lake),...",NaN,4c22665899282d7f411067b0,1
1,Food Court,Food Court,520 N Michigan Ave,at The Shops at North Bridge,41.892102,-87.626163,"[{'label': 'display', 'lat': 41.8921018, 'lng'...",1169,60611,US,Chicago,IL,United States,[520 N Michigan Ave (at The Shops at North Bri...,NaN,4b1695b8f964a52094ba23e3,2
2,Revival Food Hall,Food Court,125 S Clark St,W Adams St,41.879560,-87.630550,"[{'label': 'display', 'lat': 41.87956, 'lng': ...",816,60603,US,Chicago,IL,United States,"[125 S Clark St (W Adams St), Chicago, IL 6060...",NaN,5751af74cd102f915c2cb3de,1
3,Navy Pier Food Court,Food Court,800 E Grand Ave,Navy Pier,41.891489,-87.608547,"[{'label': 'display', 'lat': 41.89148923128558...",1481,60611,US,Chicago,IL,United States,"[800 E Grand Ave (Navy Pier), Chicago, IL 6061...",NaN,500f3f21e4b09c8ae488d96e,0
4,Bubblenet Food Court,Food Court,1200 S Lake Shore Dr,@ Shedd Aquarium,41.867925,-87.613260,"[{'label': 'display', 'lat': 41.86792457273813...",1727,60605,US,Chicago,IL,United States,"[1200 S Lake Shore Dr (@ Shedd Aquarium), Chic...",NaN,4b8aac5bf964a520f97832e3,5


4- we will review the map, using different colors for each cluster.

In [58]:
venues_map = folium.Map(location=[lat, long], zoom_start=15) # generate map centred around West toronto

# add a orange circle marker to represent West Toronto
folium.features.CircleMarker(
    location = [lat, long],
    radius=10,
    color='red',
    popup='West Toronto',
    fill = True,
    fill_color = 'orange',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Chinese restaurants as blue circle markers
for lat, lng, label, cluster in zip(food_filtered.lat, food_filtered.lng, food_filtered.name, food_filtered.cluster):
    if cluster == 0:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
    elif cluster == 1:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            fill = True,
            fill_color='red',
            fill_opacity=0.6
        ).add_to(venues_map)
    elif cluster == 2:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='purple',
            fill = True,
            fill_color='purple',
            fill_opacity=0.6
        ).add_to(venues_map)
    elif cluster == 3:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='green',
            fill = True,
            fill_color='green',
            fill_opacity=0.6
        ).add_to(venues_map)
    elif cluster == 4:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='orange',
            fill = True,
            fill_color='orange',
            fill_opacity=0.6
        ).add_to(venues_map)
    elif cluster == 5:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='black',
            fill = True,
            fill_color='Black',
            fill_opacity=1
        ).add_to(venues_map)
    elif cluster == 6:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='gray',
            fill = True,
            fill_color='gray',
            fill_opacity=0.6
        ).add_to(venues_map)
    else:
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='white',
            fill = True,
            fill_color='white',
            fill_opacity=0.6
        ).add_to(venues_map)
# display map
venues_map

#### Further analysis:


We can see that the blue cluster is the closest to the park and has several venues within it.

We can also see that, between the park and the red cluster there are several streets with no food venues. The same goes between the orange and the red cluster.

We can also see that there are almost no food venues in the black cluster but its a bit far from the park, We will investigate further upon that.

Therefore, the West and South regions of the park would be good regions for the new restaurant, specially because their closer to the park.

Let's investigate both cluster to see which food options they provide.

Exploring the orange,red and black clusters.

1- First let's explore the red cluster and maybe we could open a food venue near the center of the park.

In [59]:
#Displaying the red cluster
food_filtered.loc[food_filtered['cluster'] == 1,:]

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,cluster
0,Atrium Food Court,Food Court,"100 W Randolph, Concourse",btw Clark & Lake,41.885163,-87.631639,"[{'label': 'display', 'lat': 41.88516275046533...",911,60601,US,Chicago,IL,United States,"[100 W Randolph, Concourse (btw Clark & Lake),...",NaN,4c22665899282d7f411067b0,1
2,Revival Food Hall,Food Court,125 S Clark St,W Adams St,41.879560,-87.630550,"[{'label': 'display', 'lat': 41.87956, 'lng': ...",816,60603,US,Chicago,IL,United States,"[125 S Clark St (W Adams St), Chicago, IL 6060...",NaN,5751af74cd102f915c2cb3de,1
10,Chicago Union Station Food Court,Food Court,225 S Canal St,at Adams St.,41.879339,-87.638738,"[{'label': 'display', 'lat': 41.879339217638, ...",1474,60606,US,Chicago,IL,United States,"[225 S Canal St (at Adams St.), Chicago, IL 60...",NaN,4c489f3a19fde21e8b460c76,1
11,Bridgeport Pasty - Food Truck,Food Truck,NaN,NaN,41.879691,-87.636715,"[{'label': 'display', 'lat': 41.87969136866029...",1302,60608,US,Chicago,IL,United States,"[Chicago, IL 60608, United States]",NaN,4e8c8fc793adecfdb971a040,1
16,Ogilvie Food Court,Food Court,500 W Madison St,at Ogilvie Transportation Center,41.882125,-87.640672,"[{'label': 'display', 'lat': 41.88212517500138...",1599,60661,US,Chicago,IL,United States,[500 W Madison St (at Ogilvie Transportation C...,NaN,4d0ff93ea3d9721e414fd2fd,1
19,Mediterranean Express Food Truck,Food Truck,NaN,NaN,41.879865,-87.637026,"[{'label': 'display', 'lat': 41.879865, 'lng':...",1323,60606,US,Chicago,IL,United States,"[Chicago, IL 60606, United States]",NaN,5db72be12a8b810008f5453c,1
21,The Mart Food Court,Food Court,222 Merchandise Mart Plz,NaN,41.888869,-87.634052,"[{'label': 'display', 'lat': 41.88886897769535...",1285,60654,US,Chicago,IL,United States,"[222 Merchandise Mart Plz, Chicago, IL 60654, ...",NaN,4c1663e982a3c9b6403b00f9,1
22,Coco's southern style soul food,Southern / Soul Food Restaurant,426 S Clark St,NaN,41.876320,-87.630755,"[{'label': 'display', 'lat': 41.87632, 'lng': ...",1018,60605,US,Chicago,IL,United States,"[426 S Clark St, Chicago, IL 60605, United Sta...",NaN,4bba38e07421a5934a8bc340,1


2- let's explore the closest one to the park and see what kind of food they offer.

In [60]:
venue_id = '4c22665899282d7f411067b0' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c22665899282d7f411067b0?client_id=W4TBYFIZOQYOS0XBZOPJUNPDNFYCF42OZDP1QPIEEJZG52I4&client_secret=QFLLGATP1YPVVHGI2EZTVVV3RP54CZFGKOZR343JJKJRRKD2&v=20180604'

In [61]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto'])


{'id': '4c22665899282d7f411067b0',
 'name': 'Atrium Food Court',
 'contact': {'phone': '3123460777',
  'formattedPhone': '(312) 346-0777',
  'twitter': 'atriummall',
  'facebook': '181668594320',
  'facebookUsername': 'AtriumMallJRTC',
  'facebookName': 'Atrium at the Thompson Center'},
 'location': {'address': '100 W Randolph, Concourse',
  'crossStreet': 'btw Clark & Lake',
  'lat': 41.88516275046533,
  'lng': -87.63163902479333,
  'labeledLatLngs': [{'label': 'display',
    'lat': 41.88516275046533,
    'lng': -87.63163902479333}],
  'postalCode': '60601',
  'cc': 'US',
  'city': 'Chicago',
  'state': 'IL',
  'country': 'United States',
  'formattedAddress': ['100 W Randolph, Concourse (btw Clark & Lake)',
   'Chicago, IL 60601',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/atrium-food-court/4c22665899282d7f411067b0',
 'categories': [{'id': '4bf58dd8d48988d120951735',
   'name': 'Food Court',
   'pluralName': 'Food Courts',
   'shortName': 'Food Court',
   'icon'

From the reviews uploaded by the public we can see this indeed a Food court which offers only fast foods and food on the go so there is indeed a good option to open an Italian or Japanese restaurant. We can't be certain about vegetarian food. 

6-Let's check the black cluster because it is indeed lacking in food venues and its between the park and the IL university.

In [63]:
#Displaying the blue cluster
food_filtered.loc[food_filtered['cluster'] == 5,:]

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,cluster
4,Bubblenet Food Court,Food Court,1200 S Lake Shore Dr,@ Shedd Aquarium,41.867925,-87.61326,"[{'label': 'display', 'lat': 41.86792457273813...",1727,60605,US,Chicago,IL,United States,"[1200 S Lake Shore Dr (@ Shedd Aquarium), Chic...",NaN,4b8aac5bf964a520f97832e3,5


We can see that this is also a food court! with fast food option only and this venue is located inside the park itself so we could really benefit from opening Italian or Japanese restaurant in the black cluster area.

# Summary of the results and discussion

We can conclude that our investor have a choice of opening the restaurant nearby the park in the red cluster but suffer from major competition from the food trucks but thrive on the lack of restaurants there especially Italian or Japanese restaurant and proximity to the park.
Or our investor can open the Italian or Japanese (Not enough information about the veg menu in the restaurants) where i will lack the proximity to the park but he would enjoy the lack of competition nearby and will be on a major route from the university to the park.